<a href="https://colab.research.google.com/github/giutassarotti/IAesame/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
!pip install -r requirements.txt

#**Definizione delle funzioni per il caricamento del Dataset**#

In [186]:
import pandas

def io_load_multiple_csv(csv_path_list):
  dataframe_list = []
  for elem in csv_path_list:
    dataframe_list.append(io_load_csv(elem))
  return dataframe_list

def io_load_csv(csv_path):
  return pandas.read_csv(csv_path)

# **Definizione delle funzioni per trattare il Dataset** #

È fondamentale riconoscere quali colonne siano numeriche e quali categoriche per il trattamento dei dati.

In [187]:
import matplotlib.pyplot as pyplot
import seaborn

def is_categorical(elem):
  return type(elem) is str

def is_numeric(elem):
  return not is_categorical(elem)

def get_numeric_features(dataframe):
  numeric_features = []
  for elem in dataframe:
    if is_numeric(dataframe[elem][0]):
      numeric_features.append(elem)
  return numeric_features

def get_categorical_features(dataframe):
  categorical_features = []
  for elem in dataframe:
    if is_categorical(dataframe[elem][0]):
      categorical_features.append(elem)
  return categorical_features

#**Definizioni delle funzioni per l'Analisi del Dataset**#

Sono definite qui le funzioni per la stampa dei grafici relativi alla feature ricercata e per la stampa di informazioni generali sul Dataset.

In [188]:
def print_feature_plots(dataframe, feature_target):
  categorical_features = get_categorical_features(dataframe)
  numeric_features = get_numeric_features(dataframe)
  for elem in dataframe:
    if elem == feature_target:
      continue
    if elem in categorical_features:
      plot = seaborn.catplot(x = feature_target, 
                        col = elem, 
                        data = dataframe, 
                        kind = 'count')
    elif elem in numeric_features:
      plot = seaborn.displot(data = dataframe,
                        x = elem,
                        hue = feature_target) 
  
    pyplot.show()

def print_infos(dataframe):
  print("Informazioni del Dataframe:\n")
  print("Righe     : {}".format(dataframe.shape[0]) )
  print("Colonne  : {}".format(dataframe.shape[1]))
  print("\nFeatures :\n{}".format(dataframe.columns.tolist()))
  print("\nValori Unici :\n{}".format(dataframe.nunique()))

  print("\nInformazioni del Dataframe:")
  dataframe.info()

# **Definizioni delle funzioni per il Modello** #

Vengono qui definite le funzioni per la creazione e l'utilizzo del modello.
È stato scelto di sfruttare la libreria Scikit-learn (in breve sklearn) per definire la Pipeline e l'algoritmo di classificazione.

Come classificatore è stato scelto l'algoritmo di Random Forest, che è un algoritmo di apprendimento supervisionato di tipo Ensemble, ossia una tecnica che combina le previsioni di più algoritmi di apprendimento per fare previsioni più accurate. 

Nello specifico, utilizza il Bootstrap Aggregation (Bagging in breve) come metodo di Ensemble e il Decision Tree come modello individuale, ossia combina molti alberi decisionali in un unico modello.

Il risultato finale restituito dal Random Forest altro non è che la maggioranza della classe risultante dei diversi Decision Tree.

In [200]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder


def create_preprocessor(dataframe):
  numeric_features = get_numeric_features(dataframe)
  numeric_transformer = Pipeline(
      steps = [('imputer', SimpleImputer(strategy='median')),
               ('scaler', StandardScaler())])

  categorical_features = get_categorical_features(dataframe)
  categorical_transformer = Pipeline(
      steps=[('imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore'))])

  return ColumnTransformer(
      transformers=[('num', numeric_transformer, numeric_features),
                    ('cat', categorical_transformer, categorical_features)])

def create_model(dataframe):
  preprocessor = create_preprocessor(dataframe)

  return Pipeline(
      steps = [('preprocessor', preprocessor),
               ('classifier', RandomForestClassifier())])
  
def train_model(model, dataframe_train, dataframe_train_target):
  model.fit(dataframe_train, dataframe_train_target)
  print('Training score: {}'.format(model.score(dataframe_train, dataframe_train_target)))
  return model

def print_test_model(model, dataframe_test, dataframe_test_target):
  preds = model.predict(dataframe_test)
  print('Test score: {}'.format(model.score(dataframe_test, dataframe_test_target)))
  print(classification_report(dataframe_test_target, preds, target_names= ['Usciti', 'Rimasti']))

def print_accuracy(model, dataframe_test, dataframe_test_target):
  preds = model.predict(dataframe_test)
  print('Accuratezza:')
  print(accuracy_score(dataframe_test_target, preds))

# **Caricamento del Dataset** #



In [190]:
dataframe_train_path = "train.csv"
dataframe_test_path = "test.csv"

dataframe_train, dataframe_test = io_load_multiple_csv([dataframe_train_path, dataframe_test_path])

# **Analisi del Dataset** #




**Stampa preliminare delle prime 5 righe del Dataset**

In [ ]:
dataframe_train.head()

È necessario l'utilizzo di stampe per valutare manualmente:

*   Presenza di valori indesiderati nel Dataset, quali elementi vuoti o di tipo sbagliato all'interno, che altrimenti altererebbero il riconoscimento della tipologia di colonna (Numerale o Categorico) o creerebbero altri problemi.
*   Presenza di colonne indesiderate, quali ad esempio colonne di valori ID o di altre informazioni totalmente inutili o dannose per l'algoritmo.

**Filtraggio dei valori**

Viene qui di seguito eseguita una stampa dell'analisi delle colonne, dove è possibile notare che nessuna colonna presenta righe con valori nulli o sbagliati. Questa parte non necessita quindi accorgimenti.


In [ ]:
print_infos(dataframe_train)

**Filtraggio delle colonne irrilevanti**

Vengono qui creati e stampati grafici che prendono come ascissa sempre la feature "Exited", ossia mettono a confronto ogni altra feature con la feature che ci interessa prevedere.

Verranno inizialmente escluse le colonne inutili per la correlazione con la feature "Exited", ossia di natura completamente casuale rispetto al fatto che l'utente sia uscito o meno dalla banca:

*   CustomerId
*   RowNumber
*   Surname

Successivamente, avviando la stampa dei grafici, è possibile considerare la possibilità di ignorare una feature perchè visivamente poco rilevante per l'algoritmo.

In [ ]:
dataframe_train.pop("CustomerId")
dataframe_train.pop("Surname")
dataframe_train.pop("RowNumber")

In [ ]:
feature_target = "Exited"
print_feature_plots(dataframe_train, feature_target)

#**Esecuzione**#

####**Preparazione**####

In [196]:
le = LabelEncoder()

dataframe_train_target = le.fit_transform(dataframe_train.pop(feature_target))
dataframe_test_target = le.transform(dataframe_test.pop(feature_target))

####**Classificatore**####

In [ ]:
dataframe_train = dataframe_train.drop_duplicates()

classifier = create_model(dataframe_train)

classifier = train_model(classifier, dataframe_train, dataframe_train_target)


**Stampa Totale dei risultati**

In [202]:
print_test_model(classifier, dataframe_test, dataframe_test_target)

Test score: 0.85375
              precision    recall  f1-score   support

      Usciti       0.86      0.97      0.91      1255
     Rimasti       0.79      0.44      0.57       345

    accuracy                           0.85      1600
   macro avg       0.83      0.70      0.74      1600
weighted avg       0.85      0.85      0.84      1600



/usr/local/lib/python3.7/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


**Stampa dell'accuratezza**

In [201]:
print_accuracy(classifier, dataframe_test, dataframe_test_target)

Accuratezza:
0.85375


/usr/local/lib/python3.7/dist-packages/sklearn/compose/_column_transformer.py:430: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)
